In [3]:
import numpy as np

In [4]:

# Replace with the actual path to your file
df = pd.read_csv("C:/Users/ibrah/OneDrive/Masaüstü/all_companies_balance_statements.csv")

NameError: name 'pd' is not defined

In [ ]:
# Replace with the actual path to your file
df2 = pd.read_csv("C:/Users/ibrah/OneDrive/Masaüstü/all_companies_cashflow_statements.csv")

In [ ]:
# Replace with the actual path to your file
df3 = pd.read_csv("C:/Users/ibrah/OneDrive/Masaüstü/all_companies_income_statements.csv")

In [ ]:
# Replace with the actual path to your file
df4 = pd.read_excel("C:/Users/ibrah/OneDrive/Masaüstü/Nimbus-Fintegral Strateji/listof90year.xlsx")

In [ ]:
# First, rename columns in each DataFrame to avoid collisions (except the join keys)
df_bs = df.add_prefix('bs_')
df_cf = df2.add_prefix('cf_')
df_is = df3.add_prefix('is_')

# But restore the join keys to their original names
for col in ['symbol', 'calendarYear', 'period']:
    df_bs[col] = df[col]
    df_cf[col] = df2[col]
    df_is[col] = df3[col]

# Now merge them one by one on 'symbol', 'calendarYear', and 'period'
combined_df = df_bs.merge(df_cf, on=['symbol', 'calendarYear', 'period'], how='outer')
combined_df = combined_df.merge(df_is, on=['symbol', 'calendarYear', 'period'], how='outer')

NameError: name 'df' is not defined

In [ ]:
combined_df.dropna()



NameError: name 'combined_df' is not defined

In [ ]:
file_path="C:/Users/ibrah/OneDrive/Masaüstü/newformat.xlsx"

combined_df.to_excel(file_path, index=False)

In [ ]:
# Gross Profit / Total Assets
combined_df['gross_profit'] = combined_df['is_revenue'] - combined_df['is_costOfRevenue']
combined_df['gross_ROA'] = combined_df['gross_profit'] / combined_df['bs_totalCurrentAssets']

# EBIT / Market Price (using total stockholders equity as market price proxy)
combined_df['EBIT_to_MP'] = combined_df['is_ebitda'] / combined_df['bs_totalStockholdersEquity']

In [ ]:
combined_df['gross_ROA'] = combined_df['gross_ROA'].replace([np.inf, -np.inf], np.nan)
combined_df['EBIT_to_MP'] = combined_df['EBIT_to_MP'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Prepare a list to collect results for each year
ranked_results = []

# Loop over each row in df4 (each year)
for idx, row in df4.iterrows():
    year = row['year']
    prev_year = year - 1

    # Get the tickers for this year, drop NaNs and flatten
    tickers = row[1:].dropna().unique().tolist()

    # Filter combined_df for previous year's financials and relevant tickers
    df_year = combined_df[
        (combined_df['calendarYear'] == prev_year) &
        (combined_df['symbol'].isin(tickers))
    ].copy()

    # Only proceed if there's data
    if df_year.empty:
        continue

    # Rank by gross_ROA (higher is better)
    df_year['rank_gross_ROA'] = df_year['gross_ROA'].rank(ascending=False, method='min')

    # Rank by EBIT_to_MP (higher is better)
    df_year['rank_EBIT_to_MP'] = df_year['EBIT_to_MP'].rank(ascending=False, method='min')

    # Sum ranks
    df_year['combined_score'] = df_year['rank_gross_ROA'] + df_year['rank_EBIT_to_MP']

    # Final ranking based on combined score (lower is better)
    df_year['final_rank'] = df_year['combined_score'].rank(method='min')

    # Add current year for context
    df_year['ranking_year'] = year

    # Save result
    ranked_results.append(df_year)

# Concatenate all results into a single DataFrame
ranked_df = pd.concat(ranked_results, ignore_index=True)

# Optional: sort by year and final rank
ranked_df = ranked_df.sort_values(by=['ranking_year', 'final_rank'])

In [ ]:
ranked_df

,bs_Symbol,bs_Unnamed: 1,bs_date,bs_symbol,bs_reportedCurrency,bs_cik,bs_fillingDate,bs_acceptedDate,bs_calendarYear,bs_period,...,is_link,is_finalLink,gross_profit,gross_ROA,EBIT_to_MP,rank_gross_ROA,rank_EBIT_to_MP,combined_score,final_rank,ranking_year
137,GIS,25.0,1999-05-30,GIS,USD,40704.0,1999-08-23,1999-08-23 00:00:00,1999.0,FY,...,https://www.sec.gov/Archives/edgar/data/40704/...,https://www.sec.gov/Archives/edgar/data/40704/...,3.846800e+09,3.489161,7.380633,15.0,3.0,18.0,1.0,2000
196,CPB,25.0,1999-08-01,CPB,USD,16732.0,1999-10-12,1999-10-12 00:00:00,1999.0,FY,...,https://www.sec.gov/Archives/edgar/data/16732/...,https://www.sec.gov/Archives/edgar/data/16732/...,3.571000e+09,2.759660,6.668085,25.0,5.0,30.0,2.0,2000
147,K,24.0,1999-12-31,K,USD,55067.0,2000-03-24,2000-03-24 00:00:00,1999.0,FY,...,https://www.sec.gov/Archives/edgar/data/55067/...,https://www.sec.gov/Archives/edgar/data/55067/...,3.947100e+09,2.515358,1.674127,33.0,9.0,42.0,3.0,2000
125,EXC,25.0,1999-12-31,EXC,USD,1109357.0,1999-12-31,1999-12-30 19:00:00,1999.0,FY,...,NaN,NaN,3.291600e+09,2.714498,0.927611,27.0,16.0,43.0,4.0,2000
27,PEP,25.0,1999-12-25,PEP,USD,77476.0,2000-03-21,2000-03-21 00:00:00,1999.0,FY,...,https://www.sec.gov/Archives/edgar/data/77476/...,https://www.sec.gov/Archives/edgar/data/77476/...,1.301800e+10,3.119578,0.684784,20.0,31.0,51.0,5.0,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.sec.gov/Archives/edgar/data/113686...,https://www.sec.gov/Archives/edgar/data/113686...,1.278900e+09,NaN,NaN,NaN,NaN,NaN,NaN,2025
9881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.sec.gov/Archives/edgar/data/91576/...,https://www.sec.gov/Archives/edgar/data/91576/...,8.427000e+09,NaN,NaN,NaN,NaN,NaN,NaN,2025
9882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.sec.gov/Archives/edgar/data/746515...,https://www.sec.gov/Archives/edgar/data/746515...,1.060052e+10,NaN,NaN,NaN,NaN,NaN,NaN,2025
9883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.sec.gov/Archives/edgar/data/879101...,https://www.sec.gov/Archives/edgar/data/879101...,1.399361e+09,NaN,NaN,NaN,NaN,NaN,NaN,2025


In [ ]:
file_path="C:/Users/ibrah/OneDrive/Masaüstü/ranking.xlsx"

ranked_df.to_excel(file_path, index=False)

In [ ]:
import yfinance as yf
import pandas as pd

portfolio_returns = []

for year in range(2000, 2025):
    buy_date = f"{year}-04-01"
    sell_date = f"{year+1}-03-30"
    
    top20 = ranked_df[ranked_df['ranking_year'] == year].nsmallest(20, 'final_rank')
    tickers = top20['symbol'].dropna().unique().tolist()

    if len(tickers) == 0:
        continue

    # Download adjusted close prices
    price_data = yf.download(tickers, start=buy_date, end=sell_date)['Close']

    # Ensure price_data is a DataFrame
    if isinstance(price_data, pd.Series):
        price_data = price_data.to_frame()

    # Forward-fill to handle missing weekends/holidays
    price_data = price_data.ffill().bfill()

    # Convert buy/sell dates to Timestamps and find closest available dates
    buy_date_actual = price_data.index[price_data.index.get_indexer([pd.Timestamp(buy_date)], method='nearest')[0]]
    sell_date_actual = price_data.index[price_data.index.get_indexer([pd.Timestamp(sell_date)], method='nearest')[0]]

    # Extract prices on closest available dates
    buy_prices = price_data.loc[buy_date_actual]
    sell_prices = price_data.loc[sell_date_actual]

    # Calculate returns
    returns = (sell_prices / buy_prices) - 1
    avg_return = returns.mean()

    portfolio_returns.append({
        'year': year,
        'buy_date': str(buy_date_actual.date()),
        'sell_date': str(sell_date_actual.date()),
        'average_return': avg_return,
        'top20_tickers': tickers
    })

# Create final results DataFrame
results_df = pd.DataFrame(portfolio_returns)

[*********************100%***********************]  20 of 20 completed

1 Failed download:
['HCA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-04-01 -> 2001-03-30) (Yahoo error = "Data doesn\'t exist for startDate = 954565200, endDate = 985928400")')
[*********************100%***********************]  20 of 20 completed

2 Failed downloads:
['HCA', 'PLL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2001-04-01 -> 2002-03-30) (Yahoo error = "Data doesn\'t exist for startDate = 986101200, endDate = 1017464400")')
[*********************100%***********************]  20 of 20 completed

1 Failed download:
['HCA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2002-04-01 -> 2003-03-30) (Yahoo error = "Data doesn\'t exist for startDate = 1017637200, endDate = 1049000400")')
[*********************100%***********************]  20 of 20 completed

1 Failed download:
['HCA']: YFPricesMissingError('possibly delisted; no price da

In [ ]:
results_df

,year,buy_date,sell_date,average_return,top20_tickers
0,2000,2000-04-03,2001-03-29,0.374879,"[GIS, CPB, K, EXC, PEP, CL, HCA, KR, SLM, TROW..."
1,2001,2001-04-02,2002-03-28,0.233820,"[PLL, DLX, CPB, EHC, COP, EOG, KR, K, CL, NYT,..."
2,2002,2002-04-01,2003-03-28,-0.072632,"[YUM, DLX, GIS, HCA, EOG, SLM, K, CL, KR, MO, ..."
3,2003,2003-04-01,2004-03-29,0.421360,"[YUM, DLX, CL, EFX, KR, K, CNP, TROW, HCA, VZ,..."
4,2004,2004-04-01,2005-03-29,0.185571,"[YUM, HCA, CPB, CL, EFX, SPG, K, VZ, EOG, KR, ..."
5,2005,2005-04-01,2006-03-29,0.178355,"[PLL, YUM, HCA, SPG, FNMA, CL, EFX, CPB, PGR, ..."
6,2006,2006-04-03,2007-03-29,0.204547,"[HCA, CL, YUM, FNMA, R, KR, EFX, VZ, K, CPB, A..."
7,2007,2007-04-02,2008-03-28,0.001782,"[YUM, R, CL, KR, DRI, EFX, UPS, K, VZ, XOM, MC..."
8,2008,2008-04-01,2009-03-27,-0.296433,"[R, CLX, CL, EL, RSG, KR, CPB, DRI, VZ, EXC, S..."
9,2009,2009-04-01,2010-03-29,0.527616,"[UPS, R, CL, XOM, K, KR, DRI, CPB, SPG, PEP, E..."


In [ ]:
""" 
Scrape the stock transactions from Senator periodic filings for the last week.
Not: Bu kod, https://efdsearch.senate.gov üzerinden verileri çekmektedir.
"""

import logging
import time
import pickle
from typing import List, Optional, Any
from datetime import datetime, timedelta

import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL tanımlamaları
ROOT = 'https://efdsearch.senate.gov'
LANDING_PAGE_URL = f'{ROOT}/search/home/'
SEARCH_PAGE_URL = f'{ROOT}/search/'
REPORTS_URL = f'{ROOT}/search/report/data/'

BATCH_SIZE = 100
RATE_LIMIT_SECS = 2
PDF_PREFIX = '/search/view/paper/'
LANDING_PAGE_FAIL = 'Failed to fetch filings landing page'

REPORT_COL_NAMES = [
    'tx_date',
    'file_date',
    'last_name',
    'first_name',
    'order_type',
    'ticker',
    'asset_name',
    'tx_amount'
]

# Loglama ayarları
logging.basicConfig(level=logging.INFO, format='[%(asctime)s %(levelname)s] %(message)s')
LOGGER = logging.getLogger(__name__)


def add_rate_limit(func):
    """Decorator: Her çağrıdan önce belirli bir süre bekler."""
    def with_rate_limit(*args, **kwargs):
        time.sleep(RATE_LIMIT_SECS)
        return func(*args, **kwargs)
    return with_rate_limit


def _csrf(client: requests.Session) -> str:
    """
    Oturum için CSRF token'ını alır.
    Landing page’i ziyaret edip, form üzerinden token'ı çekip gönderir.
    """
    try:
        landing_page_response = client.get(LANDING_PAGE_URL)
    except Exception as e:
        LOGGER.error(f"Landing page alınırken hata: {e}")
        raise

    if landing_page_response.url != LANDING_PAGE_URL:
        raise Exception(LANDING_PAGE_FAIL)

    soup = BeautifulSoup(landing_page_response.text, 'lxml')
    csrf_tag = soup.find(attrs={'name': 'csrfmiddlewaretoken'})
    if not csrf_tag:
        raise ValueError("CSRF token bulunamadı.")
    form_csrf = csrf_tag.get('value')
    payload = {
        'csrfmiddlewaretoken': form_csrf,
        'prohibition_agreement': '1'
    }
    client.post(LANDING_PAGE_URL, data=payload, headers={'Referer': LANDING_PAGE_URL})

    csrftoken = client.cookies.get('csrftoken', client.cookies.get('csrf'))
    if not csrftoken:
        raise ValueError("CSRF token cookie'leri bulunamadı.")
    return csrftoken


def reports_api(client: requests.Session, offset: int, token: str) -> List[List[str]]:
    """
    Periodic transaction reports API'sini sorgular.
    Sadece son 1 haftaya ait verileri çeker.
    """
    now = datetime.now()
    last_week = now - timedelta(days=7)
    payload = {
        'start': str(offset),
        'length': str(BATCH_SIZE),
        'report_types': '[11]',
        'filer_types': '[]',
        'submitted_start_date': last_week.strftime('%m/%d/%Y 00:00:00'),
        'submitted_end_date': now.strftime('%m/%d/%Y %H:%M:%S'),
        'candidate_state': '',
        'senator_state': '',
        'office_id': '',
        'first_name': '',
        'last_name': '',
        'csrfmiddlewaretoken': token
    }
    LOGGER.info(f'Getting rows starting at offset {offset}')
    response = client.post(REPORTS_URL, data=payload, headers={'Referer': SEARCH_PAGE_URL})
    response.raise_for_status()
    json_data = response.json()
    data = json_data.get('data', [])
    return data


def senator_reports(client: requests.Session) -> List[List[str]]:
    """
    Tüm rapor sonuçlarını döner.
    Sadece son 1 haftaya ait raporlar çekilir.
    """
    token = _csrf(client)
    offset = 0
    reports = reports_api(client, offset, token)
    all_reports = []
    while reports:
        all_reports.extend(reports)
        offset += BATCH_SIZE
        reports = reports_api(client, offset, token)
    LOGGER.info(f"Toplam {len(all_reports)} rapor çekildi.")
    return all_reports


def _tbody_from_link(client: requests.Session, link: str) -> Optional[Any]:
    """
    Belirtilen linkteki sayfadan tbody elementini döner.
    Eğer bulunamazsa None döner.
    """
    url = f'{ROOT}{link}'
    response = client.get(url)
    if response.url == LANDING_PAGE_URL:
        LOGGER.info("Oturum süresi doldu. CSRF token yenileniyor.")
        _csrf(client)
        response = client.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    tbodies = soup.find_all('tbody')
    return tbodies[0] if tbodies else None


def txs_for_report(client: requests.Session, row: List[str]) -> pd.DataFrame:
    """
    Bir rapor satırından hisse alım-satım işlemleri DataFrame'i oluşturur.
    """
    if len(row) < 5:
        LOGGER.warning(f"Yetersiz sütun içeren satır: {row}")
        return pd.DataFrame()
    
    first, last, _, link_html, date_received = row[:5]
    soup_link = BeautifulSoup(link_html, 'lxml')
    a_tag = soup_link.find('a')
    if not a_tag:
        LOGGER.warning(f"Link bulunamadı in row: {row}")
        return pd.DataFrame()

    link = a_tag.get('href')
    if link.startswith(PDF_PREFIX):
        # PDF formatındaki raporlar işlenemiyor.
        return pd.DataFrame()

    tbody = _tbody_from_link(client, link)
    if not tbody:
        LOGGER.warning(f"tbody bulunamadı: {link}")
        return pd.DataFrame()

    transactions = []
    for tr in tbody.find_all('tr'):
        cols = [td.get_text(strip=True) for td in tr.find_all('td')]
        if len(cols) < 8:
            continue
        # Beklenen sütun sırası: [?, tx_date, ?, ticker, asset_name, asset_type, order_type, tx_amount, ...]
        tx_date = cols[1]
        ticker = cols[3]
        asset_name = cols[4]
        asset_type = cols[5]
        order_type = cols[6]
        tx_amount = cols[7]
        if asset_type != 'Stock' and ticker.strip() in ('--', ''):
            continue
        transactions.append([tx_date, date_received, last, first, order_type, ticker, asset_name, tx_amount])
    
    if transactions:
        return pd.DataFrame(transactions, columns=REPORT_COL_NAMES)
    else:
        return pd.DataFrame()


def main() -> pd.DataFrame:
    LOGGER.info("Client başlatılıyor.")
    client = requests.Session()
    client.get = add_rate_limit(client.get)
    client.post = add_rate_limit(client.post)
    
    reports = senator_reports(client)
    txs_frames = []
    for idx, row in enumerate(reports):
        if idx % 10 == 0:
            total = sum(frame.shape[0] for frame in txs_frames)
            LOGGER.info(f"Report #{idx} işleniyor. Şimdiye kadar {total} işlem çekildi.")
        df = txs_for_report(client, row)
        if not df.empty:
            txs_frames.append(df)
    
    if txs_frames:
        all_txs = pd.concat(txs_frames, ignore_index=True)
    else:
        all_txs = pd.DataFrame()
    
    return all_txs


if __name__ == "__main__":
    senator_txs = main()
    output_file = 'notebooks/senators.pickle'
    with open(output_file, 'wb') as f:
        pickle.dump(senator_txs, f)
    LOGGER.info(f"Veriler '{output_file}' dosyasına kaydedildi.")


In [9]:
import os
import pickle

# Çıktı dizinini tanımlayın
output_dir = 'notebooks'
output_file = os.path.join(output_dir, 'senators.pickle')

# Eğer dizin mevcut değilse oluşturun
os.makedirs(output_dir, exist_ok=True)

with open(output_file, 'wb') as f:
    pickle.dump(senator_txs, f)
LOGGER.info(f"Veriler '{output_file}' dosyasına kaydedildi.")


NameError: name 'senator_txs' is not defined

In [ ]:
import logging
import time
import pickle
from typing import List, Optional, Any
from datetime import datetime, timedelta

import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# URL tanımlamaları
ROOT = 'https://efdsearch.senate.gov'
LANDING_PAGE_URL = f'{ROOT}/search/home/'
SEARCH_PAGE_URL = f'{ROOT}/search/'
REPORTS_URL = f'{ROOT}/search/report/data/'

BATCH_SIZE = 100
RATE_LIMIT_SECS = 2
PDF_PREFIX = '/search/view/paper/'
LANDING_PAGE_FAIL = 'Failed to fetch filings landing page'

REPORT_COL_NAMES = [
    'tx_date',
    'file_date',
    'last_name',
    'first_name',
    'order_type',
    'ticker',
    'asset_name',
    'tx_amount'
]

LOGGER = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(asctime)s %(levelname)s] %(message)s')



In [ ]:
""" 
Senatör'ün isim soyismi girildiğinde geçmiş işlemlerini gösteren kod
Not: Bu kod, https://efdsearch.senate.gov üzerinden verileri çekmektedir.
"""

import logging
import time
import os
from typing import List, Optional, Any
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

ROOT = 'https://efdsearch.senate.gov'
LANDING_PAGE_URL = f'{ROOT}/search/home/'
SEARCH_PAGE_URL = f'{ROOT}/search/'
REPORTS_URL = f'{ROOT}/search/report/data/'
BATCH_SIZE = 100
RATE_LIMIT_SECS = 2
PDF_PREFIX = '/search/view/paper/'
LANDING_PAGE_FAIL = 'Failed to fetch filings landing page'
REPORT_COL_NAMES = [
    'tx_date',     # İşlem tarihi
    'file_date',   # Raporun gönderildiği tarih (filing date)
    'last_name',   # Senatör soyadı
    'first_name',  # Senatör adı
    'order_type',  # İşlem türü (alım/satım)
    'ticker',      # Hisse sembolü
    'asset_name',  # Varlık adı
    'tx_amount'    # İşlem tutarı
]
LOGGER = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(asctime)s %(levelname)s] %(message)s')

def add_rate_limit(func):
    """Her çağrıdan önce belirli bir süre bekleyen decorator."""
    def with_rate_limit(*args, **kwargs):
        time.sleep(RATE_LIMIT_SECS)
        return func(*args, **kwargs)
    return with_rate_limit
def _csrf(client: requests.Session) -> str:
    """
    Oturum için CSRF token'ını alır.
    Landing page’i ziyaret edip, form üzerinden token'ı çekip gönderir.
    """
    landing_page_response = client.get(LANDING_PAGE_URL)
    if landing_page_response.url != LANDING_PAGE_URL:
        raise Exception(LANDING_PAGE_FAIL)

    landing_page = BeautifulSoup(landing_page_response.text, 'lxml')
    csrf_tag = landing_page.find(attrs={'name': 'csrfmiddlewaretoken'})
    if not csrf_tag:
        raise ValueError("CSRF token bulunamadı.")
    form_csrf = csrf_tag.get('value')
    form_payload = {
        'csrfmiddlewaretoken': form_csrf,
        'prohibition_agreement': '1'
    }
    client.post(LANDING_PAGE_URL, data=form_payload, headers={'Referer': LANDING_PAGE_URL})

    csrftoken = client.cookies.get('csrftoken', client.cookies.get('csrf'))
    if not csrftoken:
        raise ValueError("CSRF token cookie'leri bulunamadı.")
    return csrftoken
def reports_api(client: requests.Session, offset: int, token: str,
                first_name: str = '', last_name: str = '') -> List[List[str]]:
    """
    Periodic transaction reports API'sini sorgular.
    Tüm zamanları kapsayacak şekilde 01/01/2012 tarihinden itibaren verileri çeker.
    Filtre için isteğe bağlı olarak first_name ve last_name parametreleri verilebilir.
    """
    login_data = {
        'start': str(offset),
        'length': str(BATCH_SIZE),
        'report_types': '[11]',
        'filer_types': '[]',
        'submitted_start_date': '01/01/2012 00:00:00',
        'submitted_end_date': '',  # Bitiş tarihi boş bırakılırsa mevcut tüm veriler alınır.
        'candidate_state': '',
        'senator_state': '',
        'office_id': '',
        'first_name': first_name,
        'last_name': last_name,
        'csrfmiddlewaretoken': token
    }
    LOGGER.info(f'Getting rows starting at offset {offset}')
    response = client.post(REPORTS_URL, data=login_data, headers={'Referer': SEARCH_PAGE_URL})
    response.raise_for_status()
    return response.json()['data']
def senator_reports(client: requests.Session, first_name: str = '', last_name: str = '') -> List[List[str]]:
    """
    İsteğe bağlı olarak belirli bir senatörün (first_name, last_name) verilerini ya da tüm senatörlerin
    verilerini döner.
    """
    token = _csrf(client)
    idx = 0
    reports = reports_api(client, idx, token, first_name, last_name)
    all_reports: List[List[str]] = []
    while reports:
        all_reports.extend(reports)
        idx += BATCH_SIZE
        reports = reports_api(client, idx, token, first_name, last_name)
    LOGGER.info(f"Toplam {len(all_reports)} rapor çekildi.")
    return all_reports
def _tbody_from_link(client: requests.Session, link: str) -> Optional[Any]:
    """
    Belirtilen linkteki sayfadan tbody elementini döner.
    Eğer bulunamazsa None döner.
    """
    report_url = f'{ROOT}{link}'
    report_response = client.get(report_url)
    if report_response.url == LANDING_PAGE_URL:
        LOGGER.info("Oturum süresi doldu. CSRF token yenileniyor.")
        _csrf(client)
        report_response = client.get(report_url)
    report = BeautifulSoup(report_response.text, 'lxml')
    tbodies = report.find_all('tbody')
    return tbodies[0] if tbodies else None
def txs_for_report(client: requests.Session, row: List[str]) -> pd.DataFrame:
    """
    Bir rapor satırından hisse alım-satım işlemleri DataFrame'i oluşturur.
    """
    if len(row) < 5:
        LOGGER.warning(f"Yetersiz sütun içeren satır: {row}")
        return pd.DataFrame()
    first, last, _, link_html, date_received = row[:5]
    soup_link = BeautifulSoup(link_html, 'lxml')
    a_tag = soup_link.find('a')
    if not a_tag:
        LOGGER.warning(f"Link bulunamadı in row: {row}")
        return pd.DataFrame()
    link = a_tag.get('href')
    if link.startswith(PDF_PREFIX):
        return pd.DataFrame()
    tbody = _tbody_from_link(client, link)
    if not tbody:
        LOGGER.warning(f"tbody bulunamadı: {link}")
        return pd.DataFrame()
    stocks = []
    for table_row in tbody.find_all('tr'):
        cols = [c.get_text(strip=True) for c in table_row.find_all('td')]
        if len(cols) < 8:
            continue
        tx_date = cols[1]
        ticker = cols[3]
        asset_name = cols[4]
        asset_type = cols[5]
        order_type = cols[6]
        tx_amount = cols[7]
        if asset_type != 'Stock' and ticker.strip() in ('--', ''):
            continue
        stocks.append([tx_date, date_received, last, first, order_type, ticker, asset_name, tx_amount])
    if stocks:
        return pd.DataFrame(stocks, columns=REPORT_COL_NAMES)
    else:
        return pd.DataFrame()
def main(first_name: str = '', last_name: str = '') -> pd.DataFrame:
    """
    İsteğe bağlı olarak belirli bir senatörün (first_name, last_name) verilerini ya da tüm senatörlerin
    verilerini çekip bir DataFrame oluşturur.
    """
    LOGGER.info("Client başlatılıyor.")
    client = requests.Session()
    client.get = add_rate_limit(client.get)
    client.post = add_rate_limit(client.post)
    
    reports = senator_reports(client, first_name, last_name)
    all_txs_list = []
    for i, row in enumerate(reports):
        if i % 10 == 0:
            total = sum(df.shape[0] for df in all_txs_list)
            LOGGER.info(f"Report #{i} işleniyor. Şimdiye kadar {total} işlem çekildi.")
        df = txs_for_report(client, row)
        if not df.empty:
            all_txs_list.append(df)
    if all_txs_list:
        all_txs = pd.concat(all_txs_list, ignore_index=True)
    else:
        all_txs = pd.DataFrame()
    return all_txs

In [21]:
if __name__ == "__main__":
    # Tüm senatörler için filtrelemek istemiyorsanız parametreleri boş bırakın.
    senator_txs = main(first_name='', last_name='')
    output_dir = 'notebooks'
    os.makedirs(output_dir, exist_ok=True)
    output_file = f'{output_dir}/senators.csv'
    senator_txs.to_csv(output_file, index=False)
    LOGGER.info(f"Veriler '{output_file}' dosyasına CSV formatında kaydedildi.")


[2025-03-27 17:28:17,332 INFO] Client başlatılıyor.


[2025-03-27 17:28:22,689 INFO] Getting rows starting at offset 0


KeyboardInterrupt: 

In [ ]:
senator_txs

""


In [22]:
import logging
import time
import os
from typing import List, Optional, Any
from datetime import datetime, timedelta

import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL tanımlamaları
ROOT = 'https://efdsearch.senate.gov'
LANDING_PAGE_URL = f'{ROOT}/search/home/'
SEARCH_PAGE_URL = f'{ROOT}/search/'
REPORTS_URL = f'{ROOT}/search/report/data/'

BATCH_SIZE = 100
RATE_LIMIT_SECS = 2
PDF_PREFIX = '/search/view/paper/'
LANDING_PAGE_FAIL = 'Failed to fetch filings landing page'

REPORT_COL_NAMES = [
    'tx_date',
    'file_date',
    'last_name',
    'first_name',
    'order_type',
    'ticker',
    'asset_name',
    'tx_amount'
]

LOGGER = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(asctime)s %(levelname)s] %(message)s')


def add_rate_limit(func):
    """Decorator: Her çağrıdan önce belirli bir süre bekler."""
    def with_rate_limit(*args, **kwargs):
        time.sleep(RATE_LIMIT_SECS)
        return func(*args, **kwargs)
    return with_rate_limit


def _csrf(client: requests.Session) -> str:
    """
    Oturum için CSRF token'ını alır.
    Landing page’i ziyaret edip, form üzerinden token'ı çekip gönderir.
    """
    landing_page_response = client.get(LANDING_PAGE_URL)
    if landing_page_response.url != LANDING_PAGE_URL:
        raise Exception(LANDING_PAGE_FAIL)

    landing_page = BeautifulSoup(landing_page_response.text, 'lxml')
    csrf_tag = landing_page.find(attrs={'name': 'csrfmiddlewaretoken'})
    if not csrf_tag:
        raise ValueError("CSRF token bulunamadı.")
    form_csrf = csrf_tag.get('value')
    form_payload = {
        'csrfmiddlewaretoken': form_csrf,
        'prohibition_agreement': '1'
    }
    client.post(LANDING_PAGE_URL, data=form_payload, headers={'Referer': LANDING_PAGE_URL})

    csrftoken = client.cookies.get('csrftoken', client.cookies.get('csrf'))
    if not csrftoken:
        raise ValueError("CSRF token cookie'leri bulunamadı.")
    return csrftoken


def reports_api(client: requests.Session, offset: int, token: str) -> List[List[str]]:
    """
    Periodic transaction reports API'sini sorgular.
    Sadece son 5 yıla ait verileri çeker.
    """
    now = datetime.now()
    # Son 5 yıl
    start_date = now - timedelta(days=5 * 365)
    login_data = {
        'start': str(offset),
        'length': str(BATCH_SIZE),
        'report_types': '[11]',
        'filer_types': '[]',
        'submitted_start_date': start_date.strftime('%m/%d/%Y 00:00:00'),
        'submitted_end_date': now.strftime('%m/%d/%Y %H:%M:%S'),
        'candidate_state': '',
        'senator_state': '',
        'office_id': '',
        'first_name': '',
        'last_name': '',
        'csrfmiddlewaretoken': token
    }
    LOGGER.info(f'Getting rows starting at offset {offset}')
    response = client.post(REPORTS_URL, data=login_data, headers={'Referer': SEARCH_PAGE_URL})
    response.raise_for_status()
    return response.json()['data']


def senator_reports(client: requests.Session) -> List[List[str]]:
    """
    Tüm rapor sonuçlarını döner.
    Sadece son 5 yıla ait raporlar çekilir.
    """
    token = _csrf(client)
    idx = 0
    reports = reports_api(client, idx, token)
    all_reports: List[List[str]] = []
    while reports:
        all_reports.extend(reports)
        idx += BATCH_SIZE
        reports = reports_api(client, idx, token)
    LOGGER.info(f"Toplam {len(all_reports)} rapor çekildi.")
    return all_reports


def _tbody_from_link(client: requests.Session, link: str) -> Optional[Any]:
    """
    Belirtilen linkteki sayfadan tbody elementini döner.
    Eğer bulunamazsa None döner.
    """
    report_url = f'{ROOT}{link}'
    report_response = client.get(report_url)
    if report_response.url == LANDING_PAGE_URL:
        LOGGER.info("Oturum süresi doldu. CSRF token yenileniyor.")
        _csrf(client)
        report_response = client.get(report_url)
    report = BeautifulSoup(report_response.text, 'lxml')
    tbodies = report.find_all('tbody')
    return tbodies[0] if tbodies else None


def txs_for_report(client: requests.Session, row: List[str]) -> pd.DataFrame:
    """
    Bir rapor satırından hisse alım-satım işlemleri DataFrame'i oluşturur.
    """
    if len(row) < 5:
        LOGGER.warning(f"Yetersiz sütun içeren satır: {row}")
        return pd.DataFrame()
    
    first, last, _, link_html, date_received = row[:5]
    soup_link = BeautifulSoup(link_html, 'lxml')
    a_tag = soup_link.find('a')
    if not a_tag:
        LOGGER.warning(f"Link bulunamadı in row: {row}")
        return pd.DataFrame()

    link = a_tag.get('href')
    if link.startswith(PDF_PREFIX):
        return pd.DataFrame()

    tbody = _tbody_from_link(client, link)
    if not tbody:
        LOGGER.warning(f"tbody bulunamadı: {link}")
        return pd.DataFrame()

    stocks = []
    for table_row in tbody.find_all('tr'):
        cols = [c.get_text(strip=True) for c in table_row.find_all('td')]
        if len(cols) < 8:
            continue
        tx_date = cols[1]
        ticker = cols[3]
        asset_name = cols[4]
        asset_type = cols[5]
        order_type = cols[6]
        tx_amount = cols[7]
        if asset_type != 'Stock' and ticker.strip() in ('--', ''):
            continue
        stocks.append([tx_date, date_received, last, first, order_type, ticker, asset_name, tx_amount])
    
    if stocks:
        return pd.DataFrame(stocks, columns=REPORT_COL_NAMES)
    else:
        return pd.DataFrame()


def main() -> pd.DataFrame:
    LOGGER.info("Client başlatılıyor.")
    client = requests.Session()
    client.get = add_rate_limit(client.get)
    client.post = add_rate_limit(client.post)
    
    reports = senator_reports(client)
    all_txs_list = []
    for i, row in enumerate(reports):
        if i % 10 == 0:
            total = sum(df.shape[0] for df in all_txs_list)
            LOGGER.info(f"Report #{i} işleniyor. Şimdiye kadar {total} işlem çekildi.")
        df = txs_for_report(client, row)
        if not df.empty:
            all_txs_list.append(df)
    
    if all_txs_list:
        all_txs = pd.concat(all_txs_list, ignore_index=True)
    else:
        all_txs = pd.DataFrame()
    
    return all_txs


if __name__ == "__main__":
    senator_txs = main()
    
    # Çıktının kaydedileceği klasörü oluştur
    output_dir = 'notebooks'
    os.makedirs(output_dir, exist_ok=True)
    
    # Son 5 yılın başlangıç ve bitiş yılını belirle
    now = datetime.now()
    five_years_ago = now - timedelta(days=5 * 365)
    output_file = f'{output_dir}/senators_{five_years_ago.year}_{now.year}.csv'
    
    # DataFrame'i CSV dosyasına kaydet (index sütunu eklenmez)
    senator_txs.to_csv(output_file, index=False)
    LOGGER.info(f"Veriler '{output_file}' dosyasına CSV formatında kaydedildi.")


[2025-03-27 17:39:14,748 INFO] Client başlatılıyor.
[2025-03-27 17:39:19,535 INFO] Getting rows starting at offset 0
[2025-03-27 17:39:21,904 INFO] Getting rows starting at offset 100
[2025-03-27 17:39:24,348 INFO] Getting rows starting at offset 200
[2025-03-27 17:39:26,702 INFO] Getting rows starting at offset 300
[2025-03-27 17:39:29,056 INFO] Getting rows starting at offset 400
[2025-03-27 17:39:31,413 INFO] Getting rows starting at offset 500
[2025-03-27 17:39:33,771 INFO] Getting rows starting at offset 600
[2025-03-27 17:39:36,123 INFO] Getting rows starting at offset 700
[2025-03-27 17:39:38,476 INFO] Toplam 674 rapor çekildi.
[2025-03-27 17:39:38,477 INFO] Report #0 işleniyor. Şimdiye kadar 0 işlem çekildi.
[2025-03-27 17:40:04,404 INFO] Report #10 işleniyor. Şimdiye kadar 55 işlem çekildi.
[2025-03-27 17:40:28,975 INFO] Report #20 işleniyor. Şimdiye kadar 130 işlem çekildi.
[2025-03-27 17:40:54,912 INFO] Report #30 işleniyor. Şimdiye kadar 171 işlem çekildi.
[2025-03-27 17:41